**Download sample datasets**

In [1]:
!rm -rf sample_data
!wget --no-check-certificate https://download.smartappli.eu/small.tar
!mkdir FIRE
!tar -xvf small.tar -C FIRE
!rm small.tar

--2023-04-09 12:34:45--  https://download.smartappli.eu/small.tar
Resolving download.smartappli.eu (download.smartappli.eu)... 46.105.57.169, 2001:41d0:301::20
Connecting to download.smartappli.eu (download.smartappli.eu)|46.105.57.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491589120 (469M) [application/x-tar]
Saving to: ‘small.tar’

small.tar           100%[===================>] 468.82M  11.8MB/s    in 43s     

2023-04-09 12:35:29 (11.0 MB/s) - ‘small.tar’ saved [491589120/491589120]

./
./fire/
./fire/00000000.jpg
./fire/00000001.jpg
./fire/00000003.jpg
./fire/00000004.jpg
./fire/00000005.jpg
./fire/00000006.jpg
./fire/00000007.jpg
./fire/00000008.jpg
./fire/00000009.jpg
./fire/00000010.jpg
./fire/00000011.jpg
./fire/00000012.jpg
./fire/00000013.jpg
./fire/00000014.jpg
./fire/00000015.jpg
./fire/00000016.jpg
./fire/00000017.jpg
./fire/00000018.jpg
./fire/00000019.jpg
./fire/00000020.jpg
./fire/00000021.jpg
./fire/00000022.jpg
./fire/00000023.jpg
./

**Load libraries**

In [2]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader

import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision import transforms

from torch.utils.data import DataLoader, SubsetRandomSampler

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

**Train a fire model (AlexNet)**


In [3]:
batch_size	= 1
train_split = 0.7
epochs		= 30
random		= torch.Generator().manual_seed(42)
criterion   = nn.CrossEntropyLoss()
optimizer	= optim.Adam #optim.SGD
learn_rate	= 0.001


transform   = transforms.Compose([
						transforms.RandomResizedCrop(224),
						transforms.ToTensor()
					])
dataset   = datasets.ImageFolder('FIRE',transform=transform)
indices		= list(range(len(dataset)))
# shuffle
shuffle     = torch.randint(len(dataset),(len(dataset),),generator=random)
for (i,j) in enumerate(shuffle):
	indices[i],indices[j] = indices[j],indices[i]
del shuffle #clean

trainloader = DataLoader(
						dataset,
						batch_size=batch_size,
						sampler=SubsetRandomSampler(indices[:int(len(indices)*train_split)])
					)

valloader = DataLoader(
						dataset,
						batch_size=batch_size,
						sampler=SubsetRandomSampler(indices[int(len(indices)*train_split):])
					)

fire_model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
fire_model.classifier[6] = nn.Linear(in_features=4096, out_features=3, bias=True)

fire_model.to(device)

optimizer   = optimizer(fire_model.parameters(), lr=learn_rate)

for epoch in range(epochs):
	fire_model.train(True)
	running_loss = 0.0
	counter = 0
	for (i, data) in enumerate(trainloader):
		inputs, labels = data
		inputs, labels = inputs.to(device), labels.to(device)
		# zero the parameter gradients
		optimizer.zero_grad()
		# forward + backward + optimize
		outputs = fire_model(inputs)
		loss = criterion(outputs, labels)
		loss.backward()
		optimizer.step()
		# print statistics
		running_loss += loss.item()
		counter += 1
	print(f'train [{epoch + 1}, {i + 1:5d}] loss: {running_loss / counter:.3f}')
	fire_model.train(False)
	running_loss = 0.0
	counter = 0
	for (i, data) in enumerate(valloader):
		inputs, labels = data
		inputs, labels = inputs.to(device), labels.to(device)
		# forward
		outputs = fire_model(inputs)
		loss = criterion(outputs, labels)
		# print statistics
		running_loss += loss.item()
		counter += 1
	print(f'val [{epoch + 1}, {i + 1:5d}] loss: {running_loss / counter:.3f}')
torch.save(fire_model, 'firenet.pt')


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:00<00:00, 289MB/s]
/usr/local/lib/python3.9/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train [1,   553] loss: 1.124
val [1,   238] loss: 1.101
train [2,   553] loss: 1.332
val [2,   238] loss: 1.074
train [3,   553] loss: 1.062
val [3,   238] loss: 1.071
train [4,   553] loss: 1.054
val [4,   238] loss: 1.084
train [5,   553] loss: 1.056
val [5,   238] loss: 1.075
train [6,   553] loss: 1.060
val [6,   238] loss: 1.063
train [7,   553] loss: 1.051
val [7,   238] loss: 1.066
train [8,   553] loss: 1.055
val [8,   238] loss: 1.065
train [9,   553] loss: 1.053
val [9,   238] loss: 1.066
train [10,   553] loss: 1.053
val [10,   238] loss: 1.076
train [11,   553] loss: 1.052
val [11,   238] loss: 1.062
train [12,   553] loss: 1.051
val [12,   238] loss: 1.063
train [13,   553] loss: 1.055
val [13,   238] loss: 1.069
train [14,   553] loss: 1.050
val [14,   238] loss: 1.063
train [15,   553] loss: 1.052
val [15,   238] loss: 1.062
train [16,   553] loss: 1.049
val [16,   238] loss: 1.063
train [17,   553] loss: 1.051
val [17,   238] loss: 1.062
train [18,   553] loss: 1.049
va